In [62]:
import numpy as np
import re
import pickle
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords

In [63]:
with open('tokenizer.pkl', 'rb') as tokenizer_file:
    tokenizer = pickle.load(tokenizer_file)

In [64]:
model = load_model('gru_rnn.keras')
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 25, 50)         │       500,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ (None, 25, 150)        │        90,900 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 25, 150)        │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru_1 (GRU)                     │ (None, 100)            │        75,600 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           202 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,000,108 (3.82 MB)

 Trainable params: 166,702 (651.18 KB)

 Non-trainable params: 500,000 (1.91 MB)

 Optimizer params: 333,406 (1.27 MB)

In [65]:
def load_stopwords(file_path):
    with open(file_path, 'r') as file:
        return set(file.read().splitlines())

stopwords = load_stopwords('english_stopwords.txt') 

In [66]:
def preprocess_text(text):
    # Convert to lowercase
    text = text.lower()
    
    # Handle negations more effectively
    negations = ["not", "no", "never", "none", "n't"]
    words = text.split()
    
    # Create a flag to indicate if the current word is negated
    negation_flag = False
    processed_words = []

    for word in words:
        if word in negations:
            negation_flag = True
        else:
            if negation_flag:
                processed_words.append("not_" + word)  # Prefix 'not_' to the word
                negation_flag = False  # Reset the flag after the first word
            else:
                processed_words.append(word)

    text = ' '.join(processed_words)

    # Remove URLs, mentions, hashtags, punctuation, and numbers
    text = re.sub(r'http\S+|www\S+|https\S+', '', text)
    text = re.sub(r'@\w+', '', text)
    text = re.sub(r'#', '', text)
    text = re.sub(r'[^a-z\s]', '', text)

    # Remove stopwords
    tokens = text.split()
    tokens = [word for word in tokens if word not in stopwords]
    return ' '.join(tokens)

In [78]:
sample_input = "service was not good"
preprocessed_input = preprocess_text(sample_input)

In [79]:
input_sequence = tokenizer.texts_to_sequences([preprocessed_input])
input_padded = pad_sequences(input_sequence, maxlen=60, padding='pre')
predictions = model.predict(input_padded)
predicted_class = (predictions < 0.5).astype(int)
predicted_label = 'Positive' if predicted_class[0][0] == 1 else 'Negative'
predicted_probability = predictions[0][0]

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step


In [80]:
print(f"Input: {sample_input}")
print(f"Predicted class: {predicted_label}")
print(f"Predicted probability of 'Good': {predicted_probability:.4f}")
print(f"Predicted probability of 'Bad': {1 - predicted_probability:.4f}")

Input: service was not good
Predicted class: Negative
Predicted probability of 'Good': 0.5542
Predicted probability of 'Bad': 0.4458
